## Experiment with flight data APIs

Initial goal: compare deterministic optimal (cheapest) flight search to agent-driven one

In [1]:
from utils.utils import get_flights_to_destinations, plot_price_distribution

import nest_asyncio
nest_asyncio.apply()

import json
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

from dotenv import load_dotenv
load_dotenv()

from openai import AsyncOpenAI
client = AsyncOpenAI()

from IPython.display import Markdown, display

import warnings
warnings.filterwarnings("ignore")

Test query with the function

In [2]:
flights_df = get_flights_to_destinations(
    date="2025-05-02",
    from_airport="EWR",
    to_airports=[
        "ORY",  # Orly
        "CDG",  # Charles de Gaulle
    ],
    max_stops=1,
    #trip="one-way",
    #passengers_adults=2,
)

In [ ]:
fig = plot_price_distribution(flights_df)

In [ ]:
flights_df.head(3)

### General tool-calling agent

Showcasing tool calling capabilities by assigning the above function as a tool to an OpenAI model. Tool will return with the pandas under the hood, the AI will analyze the content and provide an answer

1. Set up custom tool

In [2]:
from agents import WebSearchTool
from utils.tools import get_flights_to_destinations_tool, add_tool
from utils.prompts import system_message, user_message_template
from utils.ai import ai

2. Set up `ai` args

In [7]:
payload = {
    "agent_name": "Flight Assistant",
    "system_message": system_message,
    "user_message_template": user_message_template,
    "tools": [get_flights_to_destinations_tool, add_tool, WebSearchTool()],
    "stream": True,
    "model": "gpt-4.1",
    "format_to_md": True
}

Simple test

In [ ]:
q = 'first, tell me what 4 + 10 is using the add tool, then tell me what is the difference between declarative and imperative?'
a = await ai(q, **payload)

In [ ]:
display(Markdown(a.final_output))

In [ ]:
q = 'first, tell me what 4 + 10 is using the add tool, then search for latest sp500 news'
a = await ai(q, **payload)

Test with flight related question

In [ ]:
q = 'what is 5 plus 7, then plus 10? Then, search for direct flights from Minneapolis to San Francisco on 2025-05-02'
a = await ai(q, **payload)

In [ ]:
display(Markdown(a.final_output))

In [ ]:
a.new_items

In [ ]:
a.raw_responses

Complex question

In [43]:
model_for_complex_question = 'o1'

q = 'I want to find the best deal for flying to Madeira from Budapest any time between 2025-07-01 and 2025-07-15. I will want to spend at least 5, at most 8 days there. I will be traveling with my girlfriend. Ideally I only want to have at most 1 stop. Find the optimal flight for me. I want to have the cheapest flight possible.'

a = await ai(q, **payload.update({"model": model_for_complex_question}))

In [ ]:
display(Markdown(a.final_output))

In [ ]:
a.new_items

#### Implement conversation memory

In [ ]:
payload = {
    "agent_name": "Flight Assistant",
    "system_message": system_message,
    "user_message_template": user_message_template,
    "tools": [get_flights_to_destinations_tool, add_tool, WebSearchTool()],
    "stream": True,
    "model": "gpt-4.1",
    "format_to_md": False
}

In [35]:
async def conversation():
    conversation_history = []
    
    while True:
        q = input()
        print("\n\n=== Human ===")
        print(q)
        
        if q.lower() == "quit":
            break
            
        print("\n=== AI ===")
        
        if not conversation_history:
            # First turn
            a = await ai(q, **payload)
            conversation_history = a.to_input_list()
        else:
            # Subsequent turns
            new_input = conversation_history + [{"role": "user", "content": q}]
            a = await ai(new_input, **payload)
            conversation_history = a.to_input_list()

In [36]:
await conversation()



=== Human ===
yo

=== AI ===
Hello! How can I assist you today? If you’re interested in finding flights, getting travel tips, or have any other questions, just let me know!

=== Human ===
sup

=== AI ===
Hey! Not much—just here and ready to help with whatever you need. Are you looking for flight information, travel advice, or do you have another question in mind? Let me know how I can assist you!

=== Human ===
today?

=== AI ===
Hello! Today is April 21, 2025. How can I assist you today? 

If you'd like to check flights for today, please let me know your departure city and your destination. Or, if you have any other questions or need advice, feel free to ask!

=== Human ===
news in buadpest, hungary

=== AI ===
Web search call in progress: ws_680636495f58819281332db92d0b65cb041b3d1a515d8dd4
Web search call completed: ws_680636495f58819281332db92d0b65cb041b3d1a515d8dd4
Here are some recent news developments in Budapest, Hungary:

**International Criminal Court Inquiry into Hungary's 

### Try manual price checks

In [50]:
def find_cheapest_round_trip(
    outbound_date,
    total_travel_time_min,
    total_travel_time_max,
    outbound_date_flexibility,
    from_airport,
    to_airports,
    max_stops=1
):
    """
    Find the cheapest round trip flights based on flexible dates and travel duration.
    
    Args:
        outbound_date (str): Starting date for outbound flight in format 'YYYY-MM-DD'
        total_travel_time_min (int): Minimum number of days for the trip
        total_travel_time_max (int): Maximum number of days for the trip
        outbound_date_flexibility (int): Number of days to check after the outbound date
        from_airport (str): Departure airport code
        to_airports (list): List of destination airport codes
        max_stops (int): Maximum number of stops allowed
        
    Returns:
        tuple: (best_combination_df, all_combinations_df) where:
            - best_combination_df is a DataFrame with the cheapest round trip
            - all_combinations_df is a DataFrame with all combinations tried
    """
    
    # Convert outbound_date to datetime
    outbound_date_dt = datetime.strptime(outbound_date, "%Y-%m-%d")
    
    # Initialize results storage
    all_combinations = []
    
    # Loop through each potential outbound date with tqdm progress bar
    for outbound_offset in tqdm(range(outbound_date_flexibility + 1), desc="Checking outbound dates"):
        current_outbound_date = outbound_date_dt + timedelta(days=outbound_offset)
        current_outbound_str = current_outbound_date.strftime("%Y-%m-%d")
        
        # Get outbound flights
        outbound_flights = get_flights_to_destinations(
            date=current_outbound_str,
            from_airport=from_airport,
            to_airports=to_airports,
            max_stops=max_stops
        )
        
        if outbound_flights.empty:
            continue
            
        # Add direction indicator
        outbound_flights['direction'] = 'outbound'
        
        # Loop through each potential return date based on min/max travel time
        for travel_days in tqdm(range(total_travel_time_min, total_travel_time_max + 1), 
                               desc=f"Checking return dates for outbound {current_outbound_str}", 
                               leave=False):
            return_date = current_outbound_date + timedelta(days=travel_days)
            return_date_str = return_date.strftime("%Y-%m-%d")
            
            # Get return flights
            return_flights = get_flights_to_destinations(
                date=return_date_str,
                from_airport=to_airports[0],  # Assuming the first airport in the list is the main one
                to_airports=[from_airport],
                max_stops=max_stops
            )
            
            if return_flights.empty:
                continue
                
            # Add direction indicator
            return_flights['direction'] = 'return'
            
            # Find cheapest combination
            min_outbound_price = outbound_flights['price'].min()
            min_return_price = return_flights['price'].min()
            total_price = min_outbound_price + min_return_price
            
            # Get the cheapest outbound and return flights
            cheapest_outbound = outbound_flights.loc[outbound_flights['price'] == min_outbound_price].iloc[0]
            cheapest_return = return_flights.loc[return_flights['price'] == min_return_price].iloc[0]
            
            # Combine all flights for this date combination
            combined_flights = pd.concat([outbound_flights, return_flights])
            
            # Store the combination details
            all_combinations.append({
                'outbound_date': current_outbound_str,
                'return_date': return_date_str,
                'travel_days': travel_days,
                'total_price': total_price,
                'outbound_price': min_outbound_price,
                'return_price': min_return_price,
                'cheapest_outbound': cheapest_outbound,
                'cheapest_return': cheapest_return,
                'all_flights': combined_flights
            })
    
    # Convert to DataFrame
    if not all_combinations:
        return pd.DataFrame(), pd.DataFrame()
    
    combinations_df = pd.DataFrame([
        {
            'outbound_date': combo['outbound_date'],
            'return_date': combo['return_date'],
            'travel_days': combo['travel_days'],
            'total_price': combo['total_price'],
            'outbound_price': combo['outbound_price'],
            'return_price': combo['return_price'],
            'outbound_airline': combo['cheapest_outbound']['name'],
            'return_airline': combo['cheapest_return']['name'],
            'outbound_stops': combo['cheapest_outbound']['stops'],
            'return_stops': combo['cheapest_return']['stops'],
            'outbound_duration': combo['cheapest_outbound']['duration_hours'],
            'return_duration': combo['cheapest_return']['duration_hours'],
        }
        for combo in all_combinations
    ])
    
    # Find the cheapest overall combination
    best_combination_idx = combinations_df['total_price'].idxmin()
    best_combination = all_combinations[best_combination_idx]['all_flights']
    
    # Sort combinations by total price
    combinations_df = combinations_df.sort_values('total_price')
    
    return best_combination, combinations_df

In [ ]:
# Example usage:
best_flights, all_combinations = find_cheapest_round_trip(
    outbound_date="2025-07-10",
    total_travel_time_min=10,
    total_travel_time_max=16,
    outbound_date_flexibility=30,
    from_airport="BUD",
    to_airports=["JFK", "EWR", "LAX", "SFO"],
    max_stops=2
)

In [ ]:
all_combinations.sort_values(['total_price']).head(10)